In [6]:
import torch
from transformers import BertModel, BertTokenizer

# Model BERT text to embeddings
class TextFeatureExtractor:
    def __init__(self, model_name='bert-base-uncased'):
        self.tokenizer = BertTokenizer.from_pretrained(model_name, cache_dir="./cache")
        self.model = BertModel.from_pretrained(model_name, output_hidden_states=True, cache_dir="./cache")
        
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.model = self.model.to(device)
    
    def extract_features(self, text: str) -> torch.Tensor:
        # Tokenize input text
        inputs = self.tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
        
        # Get embeddings
        with torch.no_grad():
            outputs = self.model(**inputs)

        # Extract embeddings from the layer
        embeddings = outputs.hidden_states[-2]

        # Average embeddings of all tokens
        sentence_embedding = torch.mean(embeddings, dim=1)
        
        return sentence_embedding

    def cosine_similarity(self, a: torch.Tensor, b: torch.Tensor) -> torch.Tensor:
        return torch.nn.functional.cosine_similarity(a, b, dim=1)



In [5]:
import pandas as pd
import torch
from tqdm import tqdm

# Assuming TextFeatureExtractor is already defined and imported

# Load the CSV file
df = pd.read_csv('/home/glooma/Code/Python/ML/Hakatons/train_dataset_tag_video/baseline/train_data_categories.csv')

extractor = TextFeatureExtractor()

# Dictionary to store embeddings
embeddings = {}

# Process each row
for _, row in tqdm(df.iterrows(), total=len(df), desc="Processing videos"):
    video_id = row['video_id']
    title = row['title']
    description = row['description']
    
    # Combine title and description
    text = f"{title} {description}"
    
    # Get embedding
    embedding = extractor.extract_features(text)
    
    # Store embedding
    embeddings[video_id] = embedding

# Find most and least similar pairs
max_similarity = -1
min_similarity = 2  # Cosine similarity is always between -1 and 1
most_similar_pair = None
least_similar_pair = None

for id1, emb1 in tqdm(embeddings.items(), desc="Comparing embeddings"):
    for id2, emb2 in embeddings.items():
        if id1 != id2:
            similarity = extractor.cosine_similarity(emb1, emb2).item()
            
            if similarity > max_similarity:
                max_similarity = similarity
                most_similar_pair = (id1, id2)
            
            if similarity < min_similarity:
                min_similarity = similarity
                least_similar_pair = (id1, id2)

# Print results
print("\nMost similar pair:")
print(f"Video IDs: {most_similar_pair}")
print(f"Similarity: {max_similarity}")
print("Texts:")
print(f"1: {df[df['video_id'] == most_similar_pair[0]]['title'].values[0]} - {df[df['video_id'] == most_similar_pair[0]]['description'].values[0]}")
print(f"2: {df[df['video_id'] == most_similar_pair[1]]['title'].values[0]} - {df[df['video_id'] == most_similar_pair[1]]['description'].values[0]}")

print("\nLeast similar pair:")
print(f"Video IDs: {least_similar_pair}")
print(f"Similarity: {min_similarity}")
print("Texts:")
print(f"1: {df[df['video_id'] == least_similar_pair[0]]['title'].values[0]} - {df[df['video_id'] == least_similar_pair[0]]['description'].values[0]}")
print(f"2: {df[df['video_id'] == least_similar_pair[1]]['title'].values[0]} - {df[df['video_id'] == least_similar_pair[1]]['description'].values[0]}")

Comparing embeddings: 100%|██████████| 1049/1049 [00:31<00:00, 33.50it/s]


Most similar pair:
Video IDs: ('0ea016128113476c741eba66ecbb5f0a', 'fe8ac2d7f57582ac5665692813d76efa')
Similarity: 1.000000238418579
Texts:
1: Роман Юнусов и артист Black STAR SLAME чуть не утонули - В новом выпуске шоу «Спортивный Интерес» Рома Юнусов и популярный исполнитель SLAME под присмотром многократной призёрки чемпионатов мира будут учиться технике правильного вхождения в воду, чтобы плавать, как профи. Сухим в этот раз точно никто не уйдет, смотрите сами!
2: Роман Юнусов и артист Black STAR SLAME чуть не утонули - В новом выпуске шоу «Спортивный Интерес» Рома Юнусов и популярный исполнитель SLAME под присмотром многократной призёрки чемпионатов мира будут учиться технике правильного вхождения в воду, чтобы плавать, как профи. Сухим в этот раз точно никто не уйдет, смотрите сами!

Least similar pair:
Video IDs: ('809cf37aa35541e9c82d710ae3881b4d', '24dbccf1b42bb1498de96586c17d4957')
Similarity: 0.393150269985199
Texts:
1: Малайзия. Сколько стоит отдых? - В этом выпуске Сашу В

Processing videos: 100%|██████████| 1049/1049 [07:41<00:00,  2.27it/s]
Comparing embeddings: 100%|██████████| 1049/1049 [00:31<00:00, 33.50it/s]

Most similar pair:

Video IDs: ('0ea016128113476c741eba66ecbb5f0a', 'fe8ac2d7f57582ac5665692813d76efa')

Similarity: 1.000000238418579

Texts:

1: Роман Юнусов и артист Black STAR SLAME чуть не утонули - В новом выпуске шоу «Спортивный Интерес» Рома Юнусов и популярный исполнитель SLAME под присмотром многократной призёрки чемпионатов мира будут учиться технике правильного вхождения в воду, чтобы плавать, как профи. Сухим в этот раз точно никто не уйдет, смотрите сами!

2: Роман Юнусов и артист Black STAR SLAME чуть не утонули - В новом выпуске шоу «Спортивный Интерес» Рома Юнусов и популярный исполнитель SLAME под присмотром многократной призёрки чемпионатов мира будут учиться технике правильного вхождения в воду, чтобы плавать, как профи. Сухим в этот раз точно никто не уйдет, смотрите сами!

Least similar pair:

Video IDs: ('809cf37aa35541e9c82d710ae3881b4d', '24dbccf1b42bb1498de96586c17d4957')

Similarity: 0.393150269985199

Texts:

1: Малайзия. Сколько стоит отдых? - В этом выпуске Сашу Великолепного отправили рушить стереотипы в Малайзию. Там Саша: Неприлично много ест, но находит этому оправдание. Раскрывает секрет переулка любовниц в городе Ипо. Боится встречи с крокодилами. Становится жертвой страстного танца в секретном баре Куала-Лумпура. Знакомится с Брюсом Ли и исследует пещеру Темпурунг. И очень, очень много развлекается.  За знакомство со страной спасибо Любови (@lyuba_expat) и Марии (@travelmusha) от всей команды 🙂  За приколы в конце выпуска спасибо обстоятельствам.  #малайзия #куалалумпур #путешествия #сколькостоитотдых

2: Baikal Mile 2019 - International Festival of speed